# ai with pytorch for detecting heart disease in humans

## import dependencies

In [1]:
import pandas as pd
import numpy as np
import torch

## dataset stuff

### import dataset csv

In [ ]:
df = pd.read_csv('dataset.csv')
df = df.sample(frac=1)

#show table
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diagnosis
283,35,1,2,122,192,0,0,174,0,0.0,1,0.0,3.0,0
258,70,1,2,156,245,0,2,143,0,0.0,1,0.0,3.0,0
255,42,0,3,120,209,0,0,173,0,0.0,2,0.0,3.0,0
289,56,1,2,120,240,0,0,169,0,0.0,3,0.0,3.0,0
40,65,0,4,150,225,0,2,114,0,1.0,2,3.0,7.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,51,1,3,94,227,0,0,154,1,0.0,1,1.0,7.0,0
21,58,0,1,150,283,1,2,162,0,1.0,1,0.0,3.0,0
188,54,1,2,192,283,0,2,195,0,0.0,1,1.0,7.0,1
46,51,1,3,110,175,0,0,123,0,0.6,1,0.0,3.0,0


### train test split

In [9]:
ratio = 0.99
total_rows = df.shape[0]
train_size = int(total_rows*ratio)

# Split data into a test dataset and train dataset
train = df[0:train_size]
test = df[train_size:]

#convert to numpy arrays (so its autodiff compatible)
train_data = train.to_numpy()
test_data = test.to_numpy()

#check
print(df)

print(train)
print(test)

print(train_data)
print(test_data)

     age   sex   cp   trestbps   chol   fbs   restecg   thalach   exang  \
283   35     1    2        122    192     0         0       174       0   
258   70     1    2        156    245     0         2       143       0   
255   42     0    3        120    209     0         0       173       0   
289   56     1    2        120    240     0         0       169       0   
40    65     0    4        150    225     0         2       114       0   
..   ...   ...  ...        ...    ...   ...       ...       ...     ...   
131   51     1    3         94    227     0         0       154       1   
21    58     0    1        150    283     1         2       162       0   
188   54     1    2        192    283     0         2       195       0   
46    51     1    3        110    175     0         0       123       0   
17    54     1    4        140    239     0         0       160       0   

      oldpeak   slope   ca  thal   diagnosis  
283       0.0       1  0.0   3.0           0  
258  